## 1 Question: **Answer**

In [1]:
import sqlite3
import csv

In [ ]:
def load_bank_churn_data(input_data):
    try:
        conn = sqlite3.connect("bank_churn_data.db")
        cursor = conn.cursor()
        # Creating table
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS churn (
            customer_id PRIMARY KEY,
            credit_score TEXT,
            age INTEGER,
            geography TEXT,
            gender TEXT,
            tenure INTEGER,
            Balance FLOAT,
            is_active_member TEXT,
            estimated_salary INTEGER,
            exited INTEGER)
            """
        )
        with open(input_data, "r") as csv_file:
            csv_reader = csv.reader(csv_file)
            # skip headers
            next(csv_reader)
            for row in csv_reader:
                cursor.execute("INSERT INTO churn VALUES (?, ?, ?, ?, ?,"
                                "?, ?, ?, ?, ?)", row)
        conn.commit()
    except Exception as e:
        print(f"Error: {e}")
    finally:
        conn.close()

# Loading data
data = "/content/bank_churn.csv"
load_bank_churn_data(data)

Error: UNIQUE constraint failed: churn.customer_id


In [3]:
# Loading the extension
%load_ext sql

In [4]:
# Using the extension to connect to database
%sql sqlite:///bank_churn_data.db

In [5]:
# Default to duplicated style
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [6]:
%%sql
SELECT * FROM churn LIMIT 5;

 * sqlite:///bank_churn_data.db
Done.


customer_id,credit_score,age,geography,gender,tenure,Balance,is_active_member,estimated_salary,exited
10001,690,40,France,Female,12,84041.05,Yes,130000,1
10002,850,44,Spain,Female,8,127644.24,Yes,110000,0
10003,780,66,Germany,Female,9,71115.23,Yes,120000,0
10004,730,27,France,Male,4,136918.88,Yes,90000,1
10005,730,33,Spain,Female,7,167882.53,Yes,70000,0


## 2. Question: **Answer**

In [7]:
%%sql
SELECT
    gender,
    printf('%20s', AVG(credit_score)) AS average_credit_score
FROM churn
GROUP BY gender;

 * sqlite:///bank_churn_data.db
Done.


gender,average_credit_score
Female,758.0
Male,739.0


## 3. Question: **Answer**

In [8]:
%%sql
SELECT
    geography,
    ROUND(CAST((COUNT(CASE WHEN exited = 1 THEN 1 END) * 1.0 /
                COUNT(*)) AS REAL),2) AS percentage
FROM churn
GROUP BY geography;

 * sqlite:///bank_churn_data.db
Done.


geography,percentage
France,0.5
Germany,0.25
Spain,0.33


## 4. Question: **Answer**

In [9]:
%%sql
SELECT
    ROUND(CAST(AVG(CASE WHEN is_active_member = "Yes" THEN Balance END) -
               AVG(CASE WHEN is_active_member = "No" THEN Balance END) AS REAL), 2)
    AS difference
FROM churn;

 * sqlite:///bank_churn_data.db
Done.


difference
1189.48


## Question 5: **Answer**

In [10]:
%%sql
SELECT
    CASE
        WHEN age BETWEEN 20 AND 30 THEN '20-30'
        WHEN age BETWEEN 31 AND 40 THEN '31-40'
        ELSE 'over 40'
    END AS age_group,
    AVG(estimated_salary) AS average_salary
FROM churn
GROUP BY age_group
ORDER BY average_salary DESC;

 * sqlite:///bank_churn_data.db
Done.


age_group,average_salary
over 40,108625.0
31-40,107500.0
20-30,85000.0


## 6 Question: **Answer**

In [11]:
%%sql
SELECT
    CASE
        WHEN age BETWEEN 20 AND 30 THEN '20-30'
        WHEN age BETWEEN 31 AND 40 THEN '31-40'
        WHEN age BETWEEN 41 AND 50 THEN '41-50'
        ELSE 'over 50'
    END AS age_group,
    ROUND(AVG(Balance),2) AS average_balance
FROM churn
GROUP BY age_group
ORDER BY average_balance DESC;

 * sqlite:///bank_churn_data.db
Done.


age_group,average_balance
20-30,128246.17
31-40,127547.44
over 50,106591.55
41-50,105252.12


In [12]:
# %%sql
# DROP TABLE churn;